In [ ]:
import osmnx as ox
import os
import networkx as nx
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import random
import pandas as pd

ox.settings.cache_folder = os.path.expanduser("~/osmnx_cache")
os.makedirs(ox.settings.cache_folder, exist_ok=True)

from evacuation_layer import define_od_pairs, initial_first_edges, demand_func, reroute_all, ctm_step
from flood_layer import load_flood, map_edges_to_flood_grid, flood_to_edges
from setup import build_network, add_edge_attributes, build_edge_index, build_edges_gdf
from animation_support import animate_flood_and_congestion, animate_flood_congestion_with_EVs
from ems_routing import run_ev_vehicle

In [ ]:

DT = 5.0                     # seconds
REROUTE_INTERVAL = 60        # steps (1 minute)
T_SIM = 300
FLOOD_CLOSURE_H = 0.05

flood = load_flood("/Users/sfnolde/Documents/flood_3d_masked.npy")

bbox = (-74.02, -73.97, 40.70, 40.74)
north, south, east, west = 40.74, 40.70, -73.97, -74.02

G = build_network(north, south, east, west)
add_edge_attributes(G)

edge_index, edge_list = build_edge_index(G)
edges_gdf = build_edges_gdf(G, edge_index)

edge_cells, valid = map_edges_to_flood_grid(
    G, edge_list, bbox, flood.shape[1:]
)

flood_depth = flood_to_edges(flood, edge_cells, valid)

OD_pairs, origin_nodes, dest_nodes = define_od_pairs()
first_edge = initial_first_edges(G, OD_pairs, edge_index)
origin_demand = [demand_func for _ in OD_pairs]

K, E = len(OD_pairs), len(edge_list)
n = np.zeros((K, E))
history = []
flood_e_hist = [] 


flood_map = np.load("/Users/sfnolde/Documents/flood_3d_masked.npy")   # (T, ny, nx)
X     = np.load("/Users/sfnolde/Documents/X_3d.npy")       
Y     = np.load("/Users/sfnolde/Documents/Y_3d.npy")       

In [ ]:

for t in range(T_SIM):
    flood_idx = min(t // REROUTE_INTERVAL, flood_depth.shape[0] - 1)
    flood_e = flood_depth[t]

    if t % REROUTE_INTERVAL == 0:
        next_edge = reroute_all(FLOOD_CLOSURE_H, G, edge_index, edge_list, OD_pairs, n, flood_e)

    n = ctm_step(t, n, next_edge, flood_e, G, edge_index,
                    origin_demand, first_edge, FLOOD_CLOSURE_H, DT)

    history.append(n.sum(axis=0))
    flood_e_hist.append(flood_e.copy())


In [ ]:
anim = animate_flood_and_congestion(
    flood=flood_map,
    X=X,
    Y=Y,
    history=history,
    thr=0, 
    origin_nodes=origin_nodes,
    dest_nodes=dest_nodes,
    G=G,
    edges_gdf=edges_gdf
)

HTML(anim.to_jshtml())

In [ ]:
ev_origin = 42432000
ev_destination = 387184869

In [ ]:
path_base, m_base = run_ev_vehicle(
    origin=ev_origin,
    destination=ev_destination,
    G=G,
    edge_index=edge_index,
    edge_list=edge_list,
    history=history,         
    flood=flood,
    edge_cells=edge_cells,
    DT=DT,                    
    T_START=140,
    T_MAX=300,
    policy="baseline"
)


In [ ]:
path_exp, m_exp = run_ev_vehicle(
    origin=ev_origin,
    destination=ev_destination,
    G=G,
    edge_index=edge_index,
    edge_list=edge_list,
    history=history,        
    flood=flood,
    edge_cells=edge_cells,
    DT=DT,              
    T_START=140,
    T_MAX=300,
    policy="experimental"
)


In [ ]:
anim = animate_flood_congestion_with_EVs(
    flood=flood_map,
    X=X,
    Y=Y,
    history=history,
    path_base=path_base,
    path_exp=path_exp,
    G=G,
    edges_gdf=edges_gdf,
    origin_nodes=origin_nodes,
    dest_nodes=dest_nodes,
    ev_origin=ev_origin,
    ev_destination=ev_destination,
    thr=1,
    interval=150
)

HTML(anim.to_jshtml())


In [ ]:
DEST_NODE = 387184869
N_EV = 100

# extract nodes with coordinates
nodes = list(G.nodes(data=True))
lons = np.array([data["x"] for _, data in nodes])

# define east side
median_lon = -73.99

east_nodes = [
    n for n, data in nodes
    if data["x"] >= median_lon
    and n != DEST_NODE
]
random.seed(1000)   # critical

ev_origins = random.sample(east_nodes, N_EV)

In [ ]:
results = []
i=0

for o in ev_origins:
    i+=1
    print(i)
    try:
        path_e, m_e = run_ev_vehicle(
            origin=o,
            destination=DEST_NODE,
            G=G,
            edge_index=edge_index,
            edge_list=edge_list,
            history=history,
            flood=flood,
            edge_cells=edge_cells,
            DT=DT,
            T_START=50,
            T_MAX=300,
            policy="experimental"
        )

        path_b, m_b = run_ev_vehicle(
            origin=o,
            destination=DEST_NODE,
            G=G,
            edge_index=edge_index,
            edge_list=edge_list,
            history=history,
            flood=flood,
            edge_cells=edge_cells,
            DT=DT,
            T_START=50,
            T_MAX=300,
            policy="baseline"
        )
    except nx.NodeNotFound as NNF:
        continue

    
    results.append({
        "origin": o,
        "baseline_time": m_b["travel_time_sec"],
        "experimental_time": m_e["travel_time_sec"],
        "delta": m_b["travel_time_sec"] - m_e["travel_time_sec"],
        "baseline_arrived": m_b["arrived"],
        "experimental_arrived": m_e["arrived"]
    })
    df = pd.DataFrame(results)


In [ ]:
df_ok = df[
    (df["baseline_arrived"] == True) &
    (df["experimental_arrived"] == True)
]
baseline = df_ok["baseline_time"].values
experimental = df_ok["experimental_time"].values
from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(baseline, experimental)

print(f"Paired t-test results:")
print(f"t-statistic = {t_stat:.3f}")
print(f"p-value     = {p_value:.4g}")
print(f"Mean baseline time     = {baseline.mean():.2f} s")
print(f"Mean experimental time = {experimental.mean():.2f} s")
print(f"Mean difference (B - E)= {(baseline - experimental).mean():.2f} s")

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(df_ok["delta"], bins=20)
plt.axvline(0, color="k", linestyle="--")
plt.xlabel("Baseline − Experimental travel time (s)")
plt.ylabel("Number of EVs")
plt.title("EV routing improvement (east-side origins, arrived only)")
plt.show()

In [ ]:
node_id = 42455026   #

x = G.nodes[node_id]["x"]
y = G.nodes[node_id]["y"]

fig, ax = plt.subplots(figsize=(6, 6))

# plot network (light)
edges_gdf.plot(ax=ax, color="darkgray", linewidth=0.8, zorder=1)

# # plot node
# ax.scatter(
#     x, y,
#     s=150,
#     c="red",
#     edgecolors="black",
#     zorder=5
# )

ax.set_title("Lower Mahattan Road Network")
ax.set_axis_off()
plt.show()
